In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession \
    .builder \
    .appName("concrec-recall") \
    .getOrCreate()

24/04/02 20:35:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
rating_df = spark.read.csv(
    "../anime-data/rating.csv",
    header=True,
    inferSchema=True,
)
rating_df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [4]:
rating_df = rating_df.where("rating > 7")

In [5]:
watch_seq_df = rating_df \
    .groupBy("user_id") \
    .agg(
        collect_list(
            col("anime_id").cast("string")
        ).alias("anime_ids")
    )

In [6]:
watch_seq_df.show(10)

+-------+--------------------+
|user_id|           anime_ids|
+-------+--------------------+
|    148|[20, 81, 170, 263...|
|    463|[20, 24, 68, 102,...|
|    471| [1604, 6702, 10681]|
|    496|[227, 430, 481, 4...|
|    833|[1, 6, 19, 33, 45...|
|   1088|[226, 2923, 5262,...|
|   1238|[120, 121, 150, 2...|
|   1342|[6746, 6880, 9919...|
|   1580|[849, 853, 1535, ...|
|   1591|[48, 57, 66, 232,...|
+-------+--------------------+
only showing top 10 rows



In [7]:
watch_seq = watch_seq_df.collect()

In [9]:
watch_seq[1]

Row(user_id=463, anime_ids=['20', '24', '68', '102', '164', '170', '199', '205', '243', '245', '269', '390', '431', '512', '523', '572', '597', '790', '846', '1029', '1195', '1210', '1535', '1559', '1698', '2034', '3092', '3731', '4224', '4814', '5114', '5150', '5251', '6045', '7674', '8063', '8129', '8634', '9926', '9982', '10030', '10161', '11061', '11123', '11757', '12695', '14227', '14719', '14813', '15809', '16498', '16742', '18507', '19671', '20371', '20899', '21105', '21405', '22535', '30276'])

In [10]:
watch_seq = [r['anime_ids'] for r in watch_seq]

In [11]:
watch_seq

[['20',
  '81',
  '170',
  '263',
  '264',
  '384',
  '395',
  '481',
  '552',
  '627',
  '813',
  '859',
  '874',
  '918',
  '1313',
  '1535',
  '1575',
  '1604',
  '2001',
  '2397',
  '2904',
  '2961',
  '3972',
  '4155',
  '4224',
  '5114',
  '5258',
  '7674',
  '7817',
  '9253',
  '9863',
  '9969',
  '10030',
  '10087',
  '10162',
  '10620',
  '11061',
  '11577',
  '11741',
  '11759',
  '11843',
  '12365',
  '12859',
  '14813',
  '15335',
  '15417',
  '15809',
  '16067',
  '16498',
  '16524',
  '16894',
  '17909',
  '18115',
  '18119',
  '18179',
  '18229',
  '18295',
  '18679',
  '18689',
  '19647',
  '20507',
  '20583',
  '21327',
  '22789'],
 ['20',
  '24',
  '68',
  '102',
  '164',
  '170',
  '199',
  '205',
  '243',
  '245',
  '269',
  '390',
  '431',
  '512',
  '523',
  '572',
  '597',
  '790',
  '846',
  '1029',
  '1195',
  '1210',
  '1535',
  '1559',
  '1698',
  '2034',
  '3092',
  '3731',
  '4224',
  '4814',
  '5114',
  '5150',
  '5251',
  '6045',
  '7674',
  '8063',
  '81

In [12]:
from collections import defaultdict

# [0][0]
matrix = defaultdict(lambda: defaultdict(int))

In [15]:
# n = len(watch_seq)
# demo purpose
n = 100

for i in range(n):
    seq = watch_seq[i]
    for x in range(len(seq)):
        for y in range(x + 1, len(seq)):
            a = seq[x]
            b = seq[y]
            if a == b:
                continue

            matrix[a][b] += 1
            matrix[b][a] += 1


In [17]:
# anime id: 20
# user rated id: 20 & 1535 => 13 times

matrix['20']

defaultdict(int,
            {'81': 1,
             '170': 2,
             '263': 2,
             '264': 1,
             '384': 5,
             '395': 3,
             '481': 1,
             '552': 1,
             '627': 1,
             '813': 5,
             '859': 1,
             '874': 1,
             '918': 2,
             '1313': 1,
             '1535': 13,
             '1575': 7,
             '1604': 3,
             '2001': 4,
             '2397': 1,
             '2904': 6,
             '2961': 1,
             '3972': 1,
             '4155': 2,
             '4224': 8,
             '5114': 7,
             '5258': 3,
             '7674': 4,
             '7817': 2,
             '9253': 5,
             '9863': 1,
             '9969': 2,
             '10030': 3,
             '10087': 3,
             '10162': 3,
             '10620': 4,
             '11061': 4,
             '11577': 3,
             '11741': 3,
             '11759': 4,
             '11843': 1,
             '12365': 2,
  

In [18]:
def get_transfer_prob(vs):
    neighbours = list(vs.keys())
    total_weight = __builtin__.sum(vs.values())
    probs = [vs[k] / total_weight for k in neighbours]

    return {
        'neighbours': neighbours,
        'probs': probs,
    }

In [30]:
transfer_probs = {
    k: get_transfer_prob(v) for k, v in matrix.items()
}

In [31]:
transfer_probs["20"]

{'neighbours': ['81',
  '170',
  '263',
  '264',
  '384',
  '395',
  '481',
  '552',
  '627',
  '813',
  '859',
  '874',
  '918',
  '1313',
  '1535',
  '1575',
  '1604',
  '2001',
  '2397',
  '2904',
  '2961',
  '3972',
  '4155',
  '4224',
  '5114',
  '5258',
  '7674',
  '7817',
  '9253',
  '9863',
  '9969',
  '10030',
  '10087',
  '10162',
  '10620',
  '11061',
  '11577',
  '11741',
  '11759',
  '11843',
  '12365',
  '12859',
  '14813',
  '15335',
  '15417',
  '15809',
  '16067',
  '16498',
  '16524',
  '16894',
  '17909',
  '18115',
  '18119',
  '18179',
  '18229',
  '18295',
  '18679',
  '18689',
  '19647',
  '20507',
  '20583',
  '21327',
  '22789',
  '24',
  '68',
  '102',
  '164',
  '199',
  '205',
  '243',
  '245',
  '269',
  '390',
  '431',
  '512',
  '523',
  '572',
  '597',
  '790',
  '846',
  '1029',
  '1195',
  '1210',
  '1559',
  '1698',
  '2034',
  '3092',
  '3731',
  '4814',
  '5150',
  '5251',
  '6045',
  '8063',
  '8129',
  '8634',
  '9926',
  '9982',
  '10161',
  '111

## Entrance

In [32]:
# list of anime id
entrance_items = list(transfer_probs)

In [33]:
neighbour_sum = {
    k: __builtin__.sum(
        matrix[k].values()
    ) for k in entrance_items
}

In [34]:
neighbour_sum

{'20': 1213,
 '81': 256,
 '170': 542,
 '263': 649,
 '264': 514,
 '384': 929,
 '395': 263,
 '481': 407,
 '552': 206,
 '627': 306,
 '813': 717,
 '859': 93,
 '874': 334,
 '918': 371,
 '1313': 163,
 '1535': 3262,
 '1575': 2326,
 '1604': 883,
 '2001': 1658,
 '2397': 348,
 '2904': 2163,
 '2961': 191,
 '3972': 109,
 '4155': 194,
 '4224': 2509,
 '5114': 2509,
 '5258': 497,
 '7674': 940,
 '7817': 717,
 '9253': 2178,
 '9863': 312,
 '9969': 371,
 '10030': 550,
 '10087': 1678,
 '10162': 952,
 '10620': 1430,
 '11061': 1061,
 '11577': 904,
 '11741': 1626,
 '11759': 853,
 '11843': 262,
 '12365': 491,
 '12859': 395,
 '14813': 1183,
 '15335': 697,
 '15417': 371,
 '15809': 1557,
 '16067': 440,
 '16498': 2387,
 '16524': 428,
 '16894': 679,
 '17909': 194,
 '18115': 404,
 '18119': 63,
 '18179': 236,
 '18229': 170,
 '18295': 141,
 '18679': 1878,
 '18689': 146,
 '19647': 313,
 '20507': 1365,
 '20583': 666,
 '21327': 456,
 '22789': 477,
 '24': 1013,
 '68': 520,
 '102': 112,
 '164': 1538,
 '199': 1656,
 '205':

In [35]:
total_sum = __builtin__.sum(neighbour_sum.values())

In [36]:
entrance_probs = [
    neighbour_sum[e] / total_sum for e in entrance_items
]

In [37]:
entrance_probs

[0.001953672356421993,
 0.0004123166720890604,
 0.0008729517041885576,
 0.001045287188225782,
 0.0008278545681788165,
 0.0014962585483231918,
 0.00042359095609149564,
 0.0006555190841415922,
 0.0003317860720716658,
 0.000492847272106455,
 0.0011548088042494388,
 0.00014978691603235398,
 0.000537944408116196,
 0.000597537052129068,
 0.00026252975605670645,
 0.005253816345134824,
 0.0037462835128091974,
 0.0014221703963071888,
 0.0026703946965768053,
 0.0005604929761210665,
 0.003483753756752491,
 0.00030762689206644743,
 0.00017555670803792026,
 0.0003124587280674911,
 0.0040410255088728616,
 0.0040410255088728616,
 0.0008004741641729024,
 0.0015139752803270186,
 0.0011548088042494388,
 0.0035079129367577093,
 0.0005025109441085424,
 0.000597537052129068,
 0.0008858366001913407,
 0.002702606936583763,
 0.0015333026243311934,
 0.0023031751604974857,
 0.0017088593323691136,
 0.0014559932483144944,
 0.0026188551125656727,
 0.0013738520362967521,
 0.00042198034409114774,
 0.0007908104921708

## Deep Walk

In [38]:
import numpy as np
rng = np.random.default_rng()

def one_walk(
        length,
        entrance_items,
        entrance_probs,
        transfer_probs
    ):
    start_point = rng.choice(
        entrance_items, 1, p=entrance_probs
    )[0]
    path = [str(start_point)]

    current_point = start_point
    for _ in range(length):
        neighbours = transfer_probs[current_point]['neighbours']
        transfer_prob = transfer_probs[current_point]['probs']

        next_point = rng.choice(
            neighbours, 1, p=transfer_prob
        )[0]
        path.append(str(next_point))
        current_point = next_point
    
    return path

In [40]:
n = 500
samples = [
    one_walk(
        20,
        entrance_items=entrance_items,
        entrance_probs=entrance_probs,
        transfer_probs=transfer_probs
    ) for _ in range(n)
]

In [41]:
samples[:4]

[['3256',
  '1689',
  '16001',
  '7627',
  '13469',
  '6347',
  '1177',
  '3782',
  '12189',
  '9656',
  '934',
  '5277',
  '30300',
  '11319',
  '6324',
  '21561',
  '19319',
  '9253',
  '355',
  '8769',
  '28979'],
 ['888',
  '404',
  '7',
  '126',
  '7',
  '202',
  '1004',
  '253',
  '1399',
  '1072',
  '21329',
  '849',
  '9989',
  '10397',
  '14833',
  '9041',
  '21327',
  '18679',
  '6746',
  '11843',
  '18507'],
 ['5204',
  '5081',
  '918',
  '10793',
  '11597',
  '11979',
  '19111',
  '2167',
  '59',
  '4437',
  '392',
  '732',
  '459',
  '136',
  '2680',
  '2951',
  '12115',
  '21329',
  '798',
  '1096',
  '2312'],
 ['12115',
  '2016',
  '468',
  '1023',
  '8475',
  '397',
  '2312',
  '322',
  '2235',
  '7169',
  '7193',
  '12477',
  '3438',
  '5204',
  '3010',
  '11981',
  '9931',
  '24629',
  '27969',
  '18055',
  '5081']]

## item2vec

In [42]:
sample_df = spark.createDataFrame(
    [
        [row] for row in samples
    ],
    [
        'anime_ids'
    ]
)

In [43]:
sample_df.show()

+--------------------+
|           anime_ids|
+--------------------+
|[3256, 1689, 1600...|
|[888, 404, 7, 126...|
|[5204, 5081, 918,...|
|[12115, 2016, 468...|
|[21995, 5300, 520...|
|[2403, 1060, 322,...|
|[13663, 1535, 269...|
|[430, 2417, 936, ...|
|[28677, 13667, 10...|
|[3731, 2683, 1, 8...|
|[7858, 18897, 143...|
|[1482, 8937, 2025...|
|[20507, 20787, 17...|
|[11111, 195, 1062...|
|[2759, 27821, 290...|
|[5112, 31404, 191...|
|[1824, 114, 4280,...|
|[30276, 2389, 498...|
|[2926, 226, 572, ...|
|[1943, 25159, 114...|
+--------------------+
only showing top 20 rows



In [44]:
from pyspark.ml.feature import Word2Vec

In [45]:
item2vec = Word2Vec(
    vectorSize=5,
    maxIter=2,
    windowSize=15,
)
item2vec.setInputCol("anime_ids")
item2vec.setOutputCol("anime_ids_vec")

Word2Vec_be65f6f30266

In [46]:
model = item2vec.fit(sample_df)

24/04/02 21:49:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/04/02 21:49:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [47]:
# find the top 10 similar anime, with anime id 20
rec = model.findSynonyms("20", 10)

In [49]:
rec.show()

+-----+------------------+
| word|        similarity|
+-----+------------------+
| 1535|0.9687214493751526|
|10863| 0.966637909412384|
|  138|0.9643979668617249|
|23847| 0.962168276309967|
|  182|0.9546949863433838|
|23321|0.9521903395652771|
|21881|0.9476532340049744|
|  538|0.9460459351539612|
|28121|0.9378055930137634|
|12445|0.9374246597290039|
+-----+------------------+



In [50]:
import pandas as pd
anime_df = pd.read_csv(
    "../anime-data/anime_feature.csv"
)

def show_anime(aid):
    anime = anime_df[
        anime_df["anime_id"] == int(aid)
    ].iloc[0]

    print(
        anime["anime_id"],
        anime["name"],
        anime["japanese_title"],
        anime["genre"],
    )

In [51]:
show_anime("20")

20 Naruto ナルト Action, Comedy, Martial Arts, Shounen, Super Power


In [52]:
for row in rec.collect():
    show_anime(row.word)

1535 Death Note デスノート Mystery, Police, Psychological, Supernatural, Thriller
10863 Steins;Gate: Oukoubakko no Poriomania シュタインズ ゲート 横行跋扈のポリオマニア Sci-Fi, Thriller
138 Hunter x Hunter: Greed Island ハンターｘハンター: Greed Island Action, Adventure, Shounen, Super Power
23847 Yahari Ore no Seishun Love Comedy wa Machigatteiru. Zoku やはり俺の青春ラブコメはまちがっている。続 Comedy, Drama, Romance, School
182 Tenkuu no Escaflowne 天空のエスカフローネ Adventure, Fantasy, Mecha, Romance, Shoujo
23321 Log Horizon 2nd Season ログ・ホライズン 第2シリーズ Action, Adventure, Fantasy, Game, Magic, Shounen
21881 Sword Art Online II ソードアート・オンライン II Action, Adventure, Fantasy, Game, Romance
538 Shin Tenchi Muyou! 新・天地無用! Comedy, Harem, Romance, Sci-Fi, Shounen
28121 Dungeon ni Deai wo Motomeru no wa Machigatteiru Darou ka ダンジョンに出会いを求めるのは間違っているだろうか Action, Adventure, Comedy, Fantasy, Romance
12445 Tasogare Otome x Amnesia 黄昏乙女×アムネジア Horror, Mystery, Romance, School, Shounen, Supernatural


## Embedding

In [53]:
item_vec = model.getVectors().collect()

item_emb = {}
for item in item_vec:
    item_emb[item.word] = item.vector.toArray()

In [54]:
item_emb["20"]

array([ 0.47740102,  0.12109152,  0.23583777, -0.04314741,  0.20629019])

In [60]:
@udf(returnType="array<float>")
def build_user_emb(anime_seq):
    # if it is not found, use []
    anime_embs = [
        item_emb[aid] if aid in item_emb else [] for aid in anime_seq
    ]

    # filter out empty array anime emb
    anime_embs = list(__builtin__.filter(
        lambda l: len(l) > 0, anime_embs
    ))
    
    emb = np.mean(anime_embs, axis=0)
    return emb.tolist()

In [61]:
user_emb_df = watch_seq_df.withColumn(
    "user_emb", build_user_emb(col("anime_ids"))
)

In [62]:
user_emb_df.show()

+-------+--------------------+--------------------+
|user_id|           anime_ids|            user_emb|
+-------+--------------------+--------------------+
|    148|[20, 81, 170, 263...|[0.14422181, 0.05...|
|    463|[20, 24, 68, 102,...|[0.13752317, 0.06...|
|    471| [1604, 6702, 10681]|[0.13990729, 0.00...|
|    496|[227, 430, 481, 4...|[0.13289592, 0.04...|
|    833|[1, 6, 19, 33, 45...|[0.14506084, 0.08...|
|   1088|[226, 2923, 5262,...|[0.15512311, 0.15...|
|   1238|[120, 121, 150, 2...|[0.12934762, 0.05...|
|   1342|[6746, 6880, 9919...|[0.1379545, 0.124...|
|   1580|[849, 853, 1535, ...|[0.14429592, 7.11...|
|   1591|[48, 57, 66, 232,...|[0.10461017, 0.06...|
|   1645|[20, 59, 101, 120...|[0.15125166, 0.05...|
|   1829|[1, 226, 339, 467...|[0.12019988, 0.07...|
|   1959|[19, 199, 226, 51...|[0.14756314, 0.05...|
|   2122|[64, 65, 226, 153...|[0.19170608, 0.07...|
|   2142|             [11061]|[0.17037636, 0.03...|
|   2366|[33, 226, 356, 15...|[0.12372099, 0.05...|
|   2659|[1,

/Users/jiaronghe/anaconda3/envs/py37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jiaronghe/anaconda3/envs/py37/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Save to Redis

In [63]:
from redis import Redis

In [64]:
redis = Redis()

In [65]:
redis.hset(
    'emb', '1', '[1,2,3,4]'
)

1

In [66]:
redis.hget(
    'emb', '1'
)

b'[1,2,3,4]'

In [67]:
user_emb = user_emb_df.collect()
user_emb = {
    row.user_id: row.user_emb for row in user_emb
}

/Users/jiaronghe/anaconda3/envs/py37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jiaronghe/anaconda3/envs/py37/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/jiaronghe/anaconda3/envs/py37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jiaronghe/anaconda3/envs/py37/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/jiaronghe/anaconda3/envs/py37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jiaronghe/anaconda3/envs/py37/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  

In [68]:
user_emb

{148: [0.14422181248664856,
  0.056752365082502365,
  0.13720448315143585,
  0.16975834965705872,
  0.1088586375117302],
 463: [0.13752317428588867,
  0.06635702401399612,
  0.12807631492614746,
  0.1962825208902359,
  0.1257786601781845],
 471: [0.13990728557109833,
  0.0033566076308488846,
  0.15303054451942444,
  0.008341802284121513,
  0.24867935478687286],
 496: [0.13289591670036316,
  0.04178035259246826,
  0.14871272444725037,
  0.13945752382278442,
  0.1730310320854187],
 833: [0.14506083726882935,
  0.08068916946649551,
  0.14313556253910065,
  0.17737528681755066,
  0.11365628242492676],
 1088: [0.15512311458587646,
  0.15059149265289307,
  0.15066345036029816,
  0.18832947313785553,
  -0.0254819318652153],
 1238: [0.12934762239456177,
  0.057543814182281494,
  0.13625827431678772,
  0.16598790884017944,
  0.13563799858093262],
 1342: [0.1379545032978058,
  0.12438265979290009,
  0.15599606931209564,
  0.13165463507175446,
  0.10994384437799454],
 1580: [0.14429591596126556,


In [ ]:
def save_user_emb(embs):
    